In [66]:
import pandas as pd
import numpy as np
import io

In [67]:
def carregar_liga(caminho_arquivo):
    try:
        df = pd.read_csv(caminho_arquivo)
        if 'estilo' not in df.columns:
            df['estilo'] = 'equilibrado'
        else:
            df['estilo'] = df['estilo'].fillna('equilibrado')

        print(f'Base carregada:   {caminho_arquivo}')
        return df
    except:
        print(f'Falha ao ler o arquivo:    {caminho_arquivo}')
        return None
    

base_premier = carregar_liga('premier.csv')
base_laliga = carregar_liga('laliga.csv')
base_bundesliga = carregar_liga('bundesliga.csv')

Base carregada:   premier.csv
Base carregada:   laliga.csv
Base carregada:   bundesliga.csv


In [68]:
PESO_CONSISTENCIA = 0.15
PESO_FATOR_CASA   = 0.15
PESO_MOMENTO      = 0.05
MEDIA_MOMENTO     = 5
NUM_SIMULACOES    = 10000

In [69]:
def analisar_partida_comp(time_casa, time_fora, df):
    try:
        tc = df[df['time'] == time_casa].iloc[0]
        tf = df[df['time'] == time_fora].iloc[0]

        num_jogos_total = tc['jogos']

        num_jogos_casa = tc['jogcasa']
        num_jogos_fora = tc['jogfora']

        media_gol_jgo = df['gm'].mean() / num_jogos_total

        atk_casa_idx = (tc['gmc'] / num_jogos_casa) / media_gol_jgo
        def_fora_idx = (tf['gsf'] / num_jogos_fora) / media_gol_jgo
        xg_casa = atk_casa_idx * def_fora_idx * media_gol_jgo

        atk_fora_idx = (tf['gmf'] / num_jogos_fora) / media_gol_jgo
        def_casa_idx = (tc['gsc'] / num_jogos_casa) / media_gol_jgo
        xg_fora = atk_fora_idx * def_casa_idx * media_gol_jgo

        aprov_casa = (tc['vitorias']*3 + tc['empates']) / (tc['jogos']*3)
        aprov_fora = (tf['vitorias']*3 + tf['empates']) / (tf['jogos']*3)
        xg_casa *= (1 + (aprov_casa * PESO_CONSISTENCIA))
        xg_fora *= (1 + (aprov_fora * PESO_CONSISTENCIA))

        media_chutes = df['chutes_jg'].mean()

        fator_chute_c = tc['chutes_jg'] / media_chutes
        fator_chute_f = tf['chutes_jg'] / media_chutes
        xg_casa *= (1 + (fator_chute_c -1) * 0.20)
        xg_fora *= (1 + (fator_chute_f-1) * 0.20)

        aprov_mandante = (tc['vitcasa']*3 + tc['empcasa']) / (num_jogos_casa*3)
        aprov_visitante = (tf['vitfora']*3 + tf['empfora']) / (num_jogos_fora*3)
        xg_casa *= (1 + (aprov_mandante * PESO_FATOR_CASA))
        xg_fora *= (1 + (aprov_visitante * PESO_FATOR_CASA))

        momento_c = 1 + ((tc['perf_ult_5_jgs'] - MEDIA_MOMENTO) * PESO_MOMENTO)
        momento_f = 1 + ((tf['perf_ult_5_jgs'] - MEDIA_MOMENTO) * PESO_MOMENTO)
        xg_casa *= momento_c
        xg_fora *= momento_f

        est_c = tc['estilo']
        est_f = tf['estilo']
        mult_c, mult_f = 1.0, 1.0

        print(f'========================================')
        print(f'🏟️  DUELO: {time_casa.upper()} ({est_c}) x {time_fora.upper()} ({est_f})')

        if est_c == 'posse' and est_f == 'contra_ataque':
            mult_f += 0.15
            print('   ⚠️ TÁTICA: Contra-ataque visitante explora a Posse.')
        elif est_c == 'contra_ataque' and est_f == 'posse': 
            mult_c += 0.15
            print('   ⚠️ TÁTICA: Contra-ataque mandante explora a Posse.')
        if est_c == 'defensivo' and est_f == 'pressao':
            mult_f += 0.15
            print("   ⚠️ TÁTICA: Pressão alta visitante quebra a Retranca.")
        elif est_c == 'pressao' and est_f == 'defensivo':
            mult_c += 0.15
            print('   ⚠️ TÁTICA: Pressão alta mandante quebra a Retranca.')
        if est_c == 'defensivo' and est_f == 'contra_ataque':
            mult_c += 0.10
            print('   🛡️ TÁTICA: Retranca anula velocidade do visitante.')
        elif est_c == 'contra_ataque' and est_f == 'defensivo':
            mult_f += 0.10
            print('   🛡️ TÁTICA: Retranca visitante anula velocidade da casa.')

        xg_casa_final = xg_casa * mult_c
        xg_fora_final = xg_fora * mult_f

        gols_simu_casa = np.random.poisson(xg_casa_final, NUM_SIMULACOES)
        gols_simu_fora = np.random.poisson(xg_fora_final, NUM_SIMULACOES)

        vitorias_c = np.sum(gols_simu_casa > gols_simu_fora)
        vitorias_f = np.sum(gols_simu_fora > gols_simu_casa)
        empates = np.sum(gols_simu_casa == gols_simu_fora)

        prob_c = (vitorias_c / NUM_SIMULACOES) *100
        prob_f = (vitorias_f / NUM_SIMULACOES) *100
        prob_e = (empates / NUM_SIMULACOES) *100

        print(f'========================================')
        print(f'🏟️  PREVISÃO: {time_casa.upper()} x {time_fora.upper()}')
        print(f'========================================')
        print(f'📊 DADOS TÉCNICOS:')
        print(f'   > xG (Gols Esperados): {time_casa} {xg_casa_final:.2f} - {xg_fora_final:.2f} {time_fora}')
        print(f"   > Fase (0-10): {time_casa} nota {tc['perf_ult_5_jgs']} | {time_fora} nota {tf['perf_ult_5_jgs']}")
        
        print(f'\n🎲 PROBABILIDADES (Baseado em {NUM_SIMULACOES} simulações):')
        print(f'   🟢 Vitória {time_casa}: {prob_c:.1f}%')
        print(f'   ⚪ Empate: {prob_e:.1f}%')
        print(f'   🔴 Vitória {time_fora}: {prob_f:.1f}%')
        
        print(f'\n💡 CONCLUSÃO DA IA:')
        if prob_c > 60:
            print(f'  Favorito claro: {time_casa.upper()}')
            print(f'   (Mas cuidado: Existe {prob_f+prob_e:.1f}% de chance de tropeço)')
        elif prob_f > 60:
            print(f'   Favorito Claro: {time_fora.upper()}.')
            print(f'   (Mas cuidado: Existe {prob_c+prob_e:.1f}% de chance de tropeço)')
        else:
            print(f'   JOGO ABERTO! As chances estão divididas.')
            if abs(prob_c - prob_f) < 5:
                print('  Recomendação: Evitar apostar em vencedor (Mercado de Gols ou Empate é melhor).')


    except IndexError:
        print(f"❌ Erro: Time '{time_casa}' ou '{time_fora}' não encontrado no DataFrame.")
    except Exception as e:
        print(f"❌ Erro técnico: {e}")

In [71]:
analisar_partida_comp('bayern', 'stpauli', base_bundesliga)

🏟️  DUELO: BAYERN (posse) x STPAULI (defensivo)
🏟️  PREVISÃO: BAYERN x STPAULI
📊 DADOS TÉCNICOS:
   > xG (Gols Esperados): bayern 7.39 - 0.44 stpauli
   > Fase (0-10): bayern nota 7 | stpauli nota 2

🎲 PROBABILIDADES (Baseado em 10000 simulações):
   🟢 Vitória bayern: 99.6%
   ⚪ Empate: 0.3%
   🔴 Vitória stpauli: 0.1%

💡 CONCLUSÃO DA IA:
  Favorito claro: BAYERN
   (Mas cuidado: Existe 0.4% de chance de tropeço)
